In [1]:
# Importing the libraries
import pickle
import pandas as pd
import webbrowser
!pip install wordcloud
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from matplotlib import pyplot as plt

In [2]:
from dash.dependencies import Input, Output , State
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import os
import wordcloud
from collections import Counter
import numpy as np
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# Declaring Global variables
project_name = None
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Defining My Functions
def load_model():
    global scrappedReviews
    scrappedReviews = pd.read_csv('scrappedReviews.csv')
  
    global pickle_model
    file = open("pickle_model1.pkl", 'rb') 
    pickle_model = pickle.load(file)

    global vocab
    file = open("features1.pkl", 'rb') 
    vocab = pickle.load(file)
    
    #pie chart
    print('Loading Data......')
    temp = []
    for i in scrappedReviews['Reviews']:
        temp.append(check_review(i)[0])
    scrappedReviews['sentiment'] = temp
    
    positive = len(scrappedReviews[scrappedReviews['sentiment']==1])
    negative = len(scrappedReviews[scrappedReviews['sentiment']==0])
    
    explode = (0.1,0)  

    langs = ['Positive', 'Negative',]
    students = [positive,negative]
    colors = ['#41fc1c','red']
    plt.pie(students,explode=explode,startangle=90,colors=colors, labels = langs,autopct='%1.2f%%')
    cwd = os.getcwd()
    if 'assets' not in os.listdir(cwd):
        os.makedirs(cwd+'/assets')
    plt.savefig('assets/sentiment.png')
    
    #wordcloud
    dataset = scrappedReviews['Reviews'].to_list()
    str1 = ''
    for i in dataset:
        str1 = str1+i
    str1 = str1.lower()

    stopwords = set(STOPWORDS)
    cloud = WordCloud(width = 800, height = 400,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(str1)
    cloud.to_file("assets/wordCloud.png")
    
    #drop down
    global chart_dropdown_values
    chart_dropdown_values = {}
    for i in range(400,501):
        chart_dropdown_values[scrappedReviews['Reviews'][i]] = scrappedReviews['Reviews'][i]
    chart_dropdown_values = [{"label":key, "value":values} for key,values in chart_dropdown_values.items()]
    
def check_review(reviewText):

    #reviewText has to be vectorised, that vectorizer is not saved yet
    #load the vectorize and call transform and then pass that to model preidctor
    #load it later

    transformer = TfidfTransformer()
    loaded_vec = CountVectorizer(decode_error="replace",vocabulary=vocab)
    vectorised_review = transformer.fit_transform(loaded_vec.fit_transform([reviewText]))


    # Add code to test the sentiment of using both the model
    # 0 == negative   1 == positive
    
    return pickle_model.predict(vectorised_review)

def open_browser():
    webbrowser.open_new('http://127.0.0.1:8050/')
    
def create_app_ui():
    main_layout = html.Div(
    [
    html.H1(id='Main_title', children = "Sentiment Analysis with Insights",style={'text-align':'center'}),
    html.Hr(style={'background-color':'black'}),
    html.H2(children = "Pie Chart",style = {'text-align':'center','text-decoration':'underline'}),
    html.P([html.Img(src=app.get_asset_url('sentiment.png'),style={'width':'700px','height':'400px'})],style={'text-align':'center'}),
    html.Hr(style={'background-color':'black'}),
    html.H2(children = "WordCloud",style = {'text-align':'center','text-decoration':'underline'}),
    html.P([html.Img(src=app.get_asset_url('wordCloud.png'),style={'width':'700px','height':'400px'})],style={'text-align':'center'}),
    html.Hr(style={'background-color':'black'}),
    html.H2(children = "Select a Review",style = {'text-align':'center','text-decoration':'underline'}),
    dcc.Dropdown(
                id='Chart_Dropdown', 
                  options=chart_dropdown_values,
                  placeholder = 'Select a Review',style={'font-size':'22px','height':'70px'}
                    ),
    html.H1(children = 'Missing',id='sentiment1',style={'text-align':'center'}),
    html.Hr(style={'background-color':'black'}),
    html.H2(children = "Find Sentiment of Your Review",style = {'text-align':'center','text-decoration':'underline'}),
    dcc.Textarea(
        id = 'textarea_review',
        placeholder = 'Enter the review here.....',
        style = {'width':'100%', 'height':150,'font-size':'22px'}
        ),
    
    dbc.Button(
        children = 'FInd Review',
        id = 'button_review',
        color = 'dark',
        style= {'width':'100%'}
        ),
    
    html.H1(children = 'Missing', id='result',style={'text-align':'center'})
    
    ]    
    )
    
    return main_layout



'''
Event Handling 
When some clicks the button call my method update_app_ui

Wiring 
Object      Event    Function 
Button      Click    update_app_ui

Decorators and callbacks mechanism is a way to implment wiring in python
Input  === Arguments to your callback
Output === return of your callback 

'''

'''
@app.callback(
    Output( 'result'   , 'children'     ),
    [
    Input( 'textarea_review'    ,  'value'    )
    ]
    )
def update_app_ui(textarea_value):
    
    print("Data Type = ", str(type(textarea_value)))
    print("Value = ", str(textarea_value))

    response = check_review(textarea_value)

    if (response[0] == 0):
        result = 'Negative'
    elif (response[0] == 1 ):
        result = 'Positive'
    else:
        result = 'Unknown'

    return result
'''


@app.callback(
    Output( 'result'   , 'children'     ),
    [
    Input( 'button_review'    ,  'n_clicks')
    ],
    [
    State( 'textarea_review'  ,   'value'  )
    ]
    )
def update_app_ui_2(n_clicks, textarea_value):

    print("Data Type = ", str(type(n_clicks)))
    print("Value = ", str(n_clicks))


    print("Data Type = ", str(type(textarea_value)))
    print("Value = ", str(textarea_value))


    if (n_clicks > 0):

        response = check_review(textarea_value)
        if (response[0] == 0):
            result = 'Negative'
        elif (response[0] == 1 ):
            result = 'Positive'
        else:
            result = 'Unknown'
        
        return result
        
    else:
        return ""

@app.callback(
    Output("sentiment1", "children"),
    [Input("Chart_Dropdown", "value")])
def update_sentiment(review1):
    sentiment = []
    if review1:
        if check_review(review1)==0:
            sentiment='Negative' 
        if check_review(review1)==1:
            sentiment='Positive'
    else:
        sentiment='Missing'
    return sentiment
# Main Function to control the Flow of your Project
def main():
    print("Start of your project")
    load_model()
    open_browser()
    #update_app_ui()
    
    
    global scrappedReviews
    global project_name
    global app
    
    project_name = "Sentiment Analysis with Insights"
    #print("My project name = ", project_name)
    #print('my scrapped data = ', scrappedReviews.sample(5) )
    
    # favicon  == 16x16 icon ----> favicon.ico  ----> assests
    app.title = project_name
    app.layout = create_app_ui()
    app.run_server()
    
    
    
    print("End of my project")
    project_name = None
    scrappedReviews = None
    app = None
    
        
# Calling the main function 
if __name__ == '__main__':
    main()

Start of your project
Loading Data......
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Jul/2021 16:37:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:37:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:37:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:37:41] "GET /assets/sentiment.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:37:41] "GET /assets/wordCloud.png HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:37:41] "POST /_dash-update-component HTTP/1.1" 200 -


Data Type =  <class 'NoneType'>
Value =  None
Data Type =  <class 'NoneType'>
Value =  None
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\USER

127.0.0.1 - - [15/Jul/2021 16:37:41] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [15/Jul/2021 16:40:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/Jul/2021 16:42:22] "POST /_dash-update-component HTTP/1.1" 200 -


Data Type =  <class 'int'>
Value =  1
Data Type =  <class 'str'>
Value =  Seeing review i thought it is a good product but when i used tjis product, it turned out to be the worst of all.
